In [ ]:
import os
import pandas as pd
from tqdm import tqdm

#secrets

In [53]:
from google.colab import drive
drive.mount('/content/drive')

df_all = pd.read_csv('/content/drive/My Drive/goemotions.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
print(df_all.head())

                                                text       id  \
0                                    That game hurt.  eew5j0j   
1   >sexuality shouldn’t be a grouping category I...  eemcysk   
2     You do right, if you don't care then fuck 'em!  ed2mah1   
3                                 Man I love reddit.  eeibobj   
4  [NAME] was nowhere near them, he was by the Fa...  eda6yn6   

                author            subreddit    link_id   parent_id  \
0                Brdd9                  nrl  t3_ajis4z  t1_eew18eq   
1          TheGreen888     unpopularopinion  t3_ai4q37   t3_ai4q37   
2             Labalool          confessions  t3_abru74  t1_ed2m7g7   
3        MrsRobertshaw             facepalm  t3_ahulml   t3_ahulml   
4  American_Fascist713  starwarsspeculation  t3_ackt2f  t1_eda65q2   

    created_utc  rater_id  example_very_unclear  admiration  ...  love  \
0  1.548381e+09         1                 False           0  ...     0   
1  1.548084e+09        37               

In [35]:
%%capture
!pip install -U transformers datasets bitsandbytes accelerate

In [36]:
from transformers import pipeline
from transformers import BitsAndBytesConfig
from transformers import set_seed
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
)

set_seed(42)

pretrained_model_name = "google/gemma-2-2b"
instruction_tuned_model_name = "google/gemma-2-2b-it"


In [37]:
def load_model_and_tokenizer(
               model_name,
               model_kwargs={'quantization_config':quantization_config if torch.cuda.is_available() else None,
                             'max_length': 1024,
                             'torch_dtype':torch.float16,
                             'device_map': 'auto'}
               ):
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  tokenizer.pad_token = tokenizer.eos_token
  tokenizer.padding_side = 'left'

  model = AutoModelForCausalLM.from_pretrained(
      model_name,
      **model_kwargs,
  )

  return model, tokenizer

In [38]:
model_pretrained, tokenizer_pretrained = load_model_and_tokenizer(pretrained_model_name)
model_instruction_tuned, tokenizer_instruction_tuned = load_model_and_tokenizer(instruction_tuned_model_name)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [39]:
@torch.no_grad()
def run_model(prompt,
              model,
              tokenizer,
              apply_chat_template=False,
              generation_kwargs={
                  'min_new_tokens': 1,
                  'max_new_tokens': 25,
                  'temperature': 0.1,
                  'top_p': 0.95,
                  'do_sample': True,
              }
              ):
  set_seed(42)

  if apply_chat_template:
    prompt = tokenizer.apply_chat_template(
        [{'role': 'user', 'content': prompt}],
        tokenize=False
    )

  inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
  output_ids = model.generate(
      **inputs,
      **generation_kwargs
  )

  input_ids = inputs['input_ids'][0]
  input_len = (input_ids != tokenizer.pad_token_id).sum().item()
  tokens = output_ids[0][input_len:]
  output = tokenizer.decode(tokens, skip_special_tokens=True)
  output = output.strip().split("\n")[0]

  return output

@torch.no_grad()
def run_model_batch(prompts,
              model,
              tokenizer,
              apply_chat_template=False,
              generation_kwargs={
                  'min_new_tokens': 1,
                  'max_new_tokens': 25,
                  'temperature': 0.1,
                  'top_p': 0.95,
                  'do_sample': True
              },
              tokenizer_kwargs={
                  'padding':'longest',
              }
            ):
  set_seed(42)

  if apply_chat_template:
    prompts = [
        tokenizer.apply_chat_template(
          [{'role': 'user', 'content': prompt}],
          tokenize=False,
        ) for prompt in prompts
    ]

  inputs = tokenizer(prompts, return_tensors='pt', **tokenizer_kwargs).to(model.device)
  output_ids = model.generate(
      **inputs,
      **generation_kwargs
  )

  batch_output = []
  for i in range(len(prompts)):
    input_ids = inputs['input_ids'][i]
    input_len = (input_ids != tokenizer.pad_token_id).sum().item()
    tokens = output_ids[i][input_len:]

    text = tokenizer.decode(tokens, skip_special_tokens=True)
    text = text.strip().split("\n")[0]

    batch_output.append(text)

  return batch_output

In [ ]:
# PRETRAINED_PROMPT = "Classify the emotion of this text. Text: {input} Emotion:"

PRETRAINED_PROMPT = (
    "Sentence: \"I've never felt worse.\"\n"
    "Allowed emotions: admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral\n"
    "Emotion = sadness\n\n"

    "Sentence: \"This is the best day ever!\"\n"
    "Allowed emotions: admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral\n"
    "Emotion = joy\n\n"

    "Sentence: \"Wow! I can't believe it!\"\n"
    "Allowed emotions: admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral\n"
    "Emotion = surprise\n\n"

    "Sentence: \"That's so rude!\"\n"
    "Allowed emotions: admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral\n"
    "Emotion = anger\n\n"

    "Sentence: \"{input}\"\n"
    "Allowed emotions: admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral\n"
    "Emotion ="
)


EMOTIONS = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
    'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
    'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
    'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization',
    'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]

def get_emotion(row, emotions=EMOTIONS):
    active = [e for e in emotions if row[e] == 1]
    return active[0] if len(active) == 1 else None

# Function to map model output to closest emotion label
def map_to_emotion_label(model_output, emotions=EMOTIONS):
    model_output = model_output.lower().strip()

    # Exact match first
    if model_output in emotions:
        return model_output

    # Token-based fallback
    for emotion in emotions:
        if emotion in model_output.split():
            return emotion

    return "invalid"


df_all["emotion"] = df_all.apply(get_emotion, axis=1)

# Keep only valid, non-neutral labels
df_eval = df_all[
    df_all["emotion"].isin(EMOTIONS)
].copy()

# Sample a subset for testing (adjust sample_size as needed)
sample_size = min(500, len(df_all))  # Change to len(df_all) to test all

df_sample = df_eval.sample(n=sample_size, random_state=42)

print(f"Testing emotion classification on {len(df_sample)} samples...\n")

predictions = []
actual_labels = []
correct = 0
invalid = 0

for idx, (_, row) in enumerate(df_sample.iterrows()):
    text = row["text"]
    actual_emotion = row["emotion"]

    # Generate prediction
    model_output = run_model(PRETRAINED_PROMPT.replace("{input}", text),
                            model_pretrained, tokenizer_pretrained)
    print(f"Model Output: {model_output}")
    predicted_emotion = map_to_emotion_label(model_output)

    predictions.append(predicted_emotion)
    actual_labels.append(actual_emotion)

    if predicted_emotion == "invalid":
        invalid += 1

    elif predicted_emotion == actual_emotion:
        correct += 1

    # # Print first 10 examples
    # if idx < 10:
    print(f"Text: {text}")
    print(f"{idx}: Predicted: {predicted_emotion}, Actual: {actual_emotion}\n")

# Calculate accuracy and create results dataframe
accuracy = (correct / len(df_sample)) * 100
print(f"Accuracy: {correct}/{len(df_sample)} = {accuracy:.2f}%")

# Create results dataframe
results_df = df_sample.copy()
results_df['predicted_emotion'] = predictions
results_df['correct'] = [p == a for p, a in zip(predictions, actual_labels)]

print(f"\nCorrect predictions by emotion:")
print(results_df.groupby('emotion')['correct'].agg(['sum', 'count', 'mean']))

print(f"\nPredicted vs Actual Emotion Distribution:")
print(pd.DataFrame({
    'Predicted': predictions,
    'Actual': actual_labels
}).value_counts().head(10))


Testing emotion classification on 500 samples...

Model Output: relief
Text: I will tell my NP he may have inspired pancakes in someone's future :)
0: Predicted: relief, Actual: neutral

Model Output: confusion
Text: What? I didn’t understand a thing of what you said.
1: Predicted: confusion, Actual: confusion

Model Output: disgust
Text: *Just look at the opposing team's hooker and lick your lips as every scrum starts.*
2: Predicted: disgust, Actual: neutral

Model Output: anger
Text: Meanwhile... [NAME], [NAME], [NAME], [NAME], [NAME], [NAME], [NAME], [NAME] and [NAME] don't benefit at all from playing with better players
3: Predicted: anger, Actual: realization

Model Output: anger
Text: Taking this way farther than needed...
4: Predicted: anger, Actual: neutral

Model Output: anger
Text: what i imagine you sounded like when that cursor started freaking out
5: Predicted: anger, Actual: annoyance

Model Output: anger
Text: If anything makes me belive someone DOESN'T get laid, it's co

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(
    actual_labels,
    predictions,
    labels=[
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
    'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
    'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
    'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization',
    'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]
))

In [ ]:
pd.crosstab(
    pd.Series(actual_labels, name="Actual"),
    pd.Series(predictions, name="Predicted"),
    normalize="index"
)

In [47]:
# PRETRAINED_PROMPT = "Classify the emotion of this text. Text: {input} Emotion:"

PRETRAINED_PROMPT = (
    "Sentence: \"I've never felt worse.\"\n"
    "Allowed emotions: joy, surprise, anger, sadness, disgust, fear\n"
    "Emotion = sadness\n\n"

    "Sentence: \"This is the best day ever!\"\n"
    "Allowed emotions: joy, surprise, anger, sadness, disgust, fear\n"
    "Emotion = joy\n\n"

    "Sentence: \"Wow! I can't believe it!\"\n"
    "Allowed emotions: joy, surprise, anger, sadness, disgust, fear\n"
    "Emotion = surprise\n\n"

    "Sentence: \"That's so rude!\"\n"
    "Allowed emotions: joy, surprise, anger, sadness, disgust, fear\n"
    "Emotion = anger\n\n"

    "Sentence: \"{input}\"\n"
    "Allowed emotions: joy, surprise, anger, sadness, disgust, fear\n"
    "Emotion ="
)


EMOTIONS = [
    "anger", "disgust", "fear",
    "joy", "sadness", "surprise", "neutral"
]

def get_emotion(row, emotions=EMOTIONS):
    active = [e for e in emotions if row[e] == 1]
    return active[0] if len(active) == 1 else None

# Function to map model output to closest emotion label
def map_to_emotion_label(model_output, emotions=EMOTIONS):
    model_output = model_output.lower().strip()

    # Exact match first
    if model_output in emotions:
        return model_output

    # Token-based fallback
    for emotion in emotions:
        if emotion in model_output.split():
            return emotion

    return "invalid"


df_all["emotion"] = df_all.apply(get_emotion, axis=1)

# Keep only valid, non-neutral labels
df_eval = df_all[
    df_all["emotion"].isin(EMOTIONS)
].copy()

# Sample a subset for testing (adjust sample_size as needed)
sample_size = min(500, len(df_all))  # Change to len(df_all) to test all

df_sample = df_eval.sample(n=sample_size, random_state=42)

print(f"Testing emotion classification on {len(df_sample)} samples...\n")

pretrained_predictions = []
pretrained_actual_labels = []
correct = 0
invalid = 0

for idx, (_, row) in enumerate(df_sample.iterrows()):
    text = row["text"]
    actual_emotion = row["emotion"]

    # Generate prediction
    model_output = run_model(PRETRAINED_PROMPT.replace("{input}", text),
                            model_instruction_tuned, tokenizer_instruction_tuned)
    print(f"Model Output: {model_output}")
    predicted_emotion = map_to_emotion_label(model_output)

    pretrained_predictions.append(predicted_emotion)
    pretrained_actual_labels.append(actual_emotion)

    if predicted_emotion == "invalid":
        invalid += 1

    elif predicted_emotion == actual_emotion:
        correct += 1

    # # Print first 10 examples
    # if idx < 10:
    print(f"Text: {text}")
    print(f"{idx}: Predicted: {predicted_emotion}, Actual: {actual_emotion}\n")

# Calculate accuracy and create results dataframe
accuracy = (correct / len(df_sample)) * 100
print(f"Accuracy: {correct}/{len(df_sample)} = {accuracy:.2f}%")

# Create results dataframe
results_df = df_sample.copy()
results_df['predicted_emotion'] = pretrained_predictions
results_df['correct'] = [p == a for p, a in zip(pretrained_predictions, pretrained_actual_labels)]

print(f"\nCorrect predictions by emotion:")
print(results_df.groupby('emotion')['correct'].agg(['sum', 'count', 'mean']))

print(f"\nPredicted vs Actual Emotion Distribution:")
print(pd.DataFrame({
    'Predicted': pretrained_predictions,
    'Actual': pretrained_actual_labels
}).value_counts().head(10))


Testing emotion classification on 500 samples...



/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py:1733: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Model Output: sadness
Text: You are the parent your grandson deserves. Your daughter will regret the pain she has caused.
0: Predicted: sadness, Actual: sadness

Model Output: joy
Text: Congrats for coming out 😁😁
1: Predicted: joy, Actual: joy

Model Output: sadness
Text: >the satisfaction of knowing how bent out of shape you're getting What a sad existence.
2: Predicted: sadness, Actual: sadness

Model Output: fear
Text: So..the head [NAME], is afraid of being rammed?
3: Predicted: fear, Actual: neutral

Model Output: disgust
Text: Her upper lip always looks terrible - such an easy fix, can’t believe she is so vain and never bothers to wax 
4: Predicted: disgust, Actual: joy

Model Output: anger
Text: Get your mom professional help. Any grow person who runs away from their own home afterargument definitely has mental health issues
5: Predicted: anger, Actual: neutral

Model Output: anger
Text: Don't confused sex with gender. Gender identity can't be boiled down to beards and balls.
6:

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(
    pretrained_actual_labels,
    pretrained_predictions,
    labels=["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
))